# Разведочный анализ данных и разметка

### Шаг 0: Подготовка


### Шаг 1: Импорт библиотек и настройка

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import os
import io
import requests
import json
import kagglehub
from io import StringIO
from collections import Counter

# Настройка отображения
pd.set_option('display.max_colwidth', 200)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Библиотеки загружены")
print("Текущая рабочая директория:", os.getcwd())

Библиотеки загружены
Текущая рабочая директория: /content


### Шаг 2: Загрузка и первичный анализ данных

In [14]:
from kagglehub import KaggleDatasetAdapter

# Указываем конкретный файл для загрузки
file_path = "contracts_dataset_filtered.json"

# Load the latest version
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "aldarovalexander/contracts",
    file_path,
)

print("First 5 records:", df.head())

/tmp/ipython-input-2806437580.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'contracts' dataset.
First 5 records:                 regNum  \
0   166300031414000000   
1   324300007314000000   
2   366300032914000000   
3  1010501417722000000   
4  1010501746721000064   

                                                                                                                                                                                       contractSubjectFull  \
0  1.1. Подрядчик обязуется выполнить работы по ремонту кровли и утеплению труб жилого дома №6, расположенного по адресу: Тульская область, Ленинский район, п. Молодежный, ул. Центральная (далее - об...   
1  Исполнитель обязуется выполнить работу согласно Приложению № к Контракту и сдать ее результат Заказчику, а Заказчик обязуется принять результат работы и оплатить его. Качество работы Качество выпо...   
2  1.1. По настоящему контракту поставщик обязуется поставить и передать заказчику в срок, предусмотренный настоящим контрактом, лекарственные п

In [13]:
df.shape

(200057, 3)

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Указываем конкретный файл для загрузки
file_path = "contracts_dataset_filtered.json"

# Load the latest version
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "aldarovalexander/contracts",
    file_path,
)

print("First 5 records:", df.head())

### Шаг 3: Предобработка текста

In [15]:
# Объединение текстовых полей
df['full_text'] = df['contractSubject'].fillna('') + '. ' + df['contractSubjectFull'].fillna('')

print("Длина текстов после объединения:")
print(df['full_text'].str.len().describe())

KeyError: 'contractSubject'

In [ ]:
# Базовая очистка текста
def clean_text(text):
    if pd.isna(text) or text == '':
        return ""
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление специальных символов
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s\.]', ' ', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text)
    # Удаление множественных точек
    text = re.sub(r'\.{2,}', '.', text)
    return text.strip()

df['text_clean'] = df['full_text'].apply(clean_text)

print("Пример очистки текста:")
print("ДО:", df['full_text'].iloc[0][:200])
print("ПОСЛЕ:", df['text_clean'].iloc[0][:200])

In [ ]:
# Установка библиотек для продвинутой обработки (выполнить в терминале)
# !pip install pymorphy2 nltk

In [ ]:
import pymorphy2
import nltk
from nltk.corpus import stopwords

# Скачивание стоп-слов
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

# Доменные стоп-слова
domain_stopwords = ['контракт', 'договор', 'приложение', 'пункт', 'статья', 
                   'далее', 'согласно', 'также', 'например', 'иной', 'другой',
                   'обязан', 'обязана', 'обязаны', 'обязано', 'условие', 'следующий']
custom_stopwords = set(russian_stopwords + domain_stopwords)

def advanced_text_processing(text):
    if not text:
        return ""
    
    # Токенизация
    tokens = text.split()
    
    # Удаление стоп-слов
    tokens = [token for token in tokens if token not in custom_stopwords]
    
    # Лемматизация
    morph = pymorphy2.MorphAnalyzer()
    lemmatized_tokens = []
    for token in tokens:
        try:
            lemma = morph.parse(token)[0].normal_form
            lemmatized_tokens.append(lemma)
        except:
            lemmatized_tokens.append(token)
    
    return ' '.join(lemmatized_tokens)

# Применяем к данным
print("Начата продвинутая обработка текста...")
df['text_processed'] = df['text_clean'].apply(advanced_text_processing)
print("Обработка завершена!")

print("\nПример продвинутой обработки:")
print("ДО:", df['text_clean'].iloc[0][:150])
print("ПОСЛЕ:", df['text_processed'].iloc[0][:150])

### Шаг 4: РАЗМЕТКА ДАННЫХ

In [ ]:
print("=== АНАЛИЗ СУЩЕСТВУЮЩЕЙ РАЗМЕТКИ ===")
print(f"Всего записей: {len(df)}")
print(f"Записей с OKPD2: {df['OKPD2_codes'].notna().sum()}")
print(f"Процент размеченных данных: {df['OKPD2_codes'].notna().mean():.2%}")

# Анализ структуры OKPD2 кодов
df['okpd_count'] = df['OKPD2_codes'].apply(lambda x: len(x) if x else 0)
print("\nРаспределение количества кодов OKPD2 на запись:")
print(df['okpd_count'].value_counts().sort_index())

In [ ]:
# Извлечение и нормализация целевых меток
def extract_okpd_label(okpd_codes):
    """Извлекает основной код OKPD2 для классификации"""
    if not okpd_codes or len(okpd_codes) == 0:
        return None
    
    # Берем первый код из списка
    primary_code = okpd_codes[0]
    
    # Определяем уровень детализации (начинаем с 2-значного кода)
    if len(primary_code) >= 2:
        return primary_code[:2]  # Первые 2 цифры - раздел
    else:
        return primary_code

# Создаем целевую переменную
df['target'] = df['OKPD2_codes'].apply(extract_okpd_label)

print("Уровень детализации меток:")
print(df['target'].str.len().value_counts().sort_index())

In [ ]:
# Анализ качества разметки
print("=== АНАЛИЗ КАЧЕСТВА РАЗМЕТКИ ===")

# Смотрим примеры текстов для разных классов
sample_classes = df['target'].value_counts().head(3).index

for class_label in sample_classes:
    class_texts = df[df['target'] == class_label]['text_clean'].head(2)
    print(f"\n--- Класс {class_label} ({len(df[df['target'] == class_label])} примеров) ---")
    for i, text in enumerate(class_texts):
        print(f"{i+1}. {text[:150]}...")

In [ ]:
# Обработка проблем разметки
missing_target = df['target'].isna().sum()
print(f"Записей без меток: {missing_target}")

# Анализ малочисленных классов
class_counts = df['target'].value_counts()
small_classes = class_counts[class_counts < 5]
print(f"Малочисленные классы (менее 5 примеров): {len(small_classes)}")

# Решение: удаляем записи без меток и малочисленные классы
df_labeled = df[df['target'].notna()].copy()
df_labeled = df_labeled[~df_labeled['target'].isin(small_classes.index)]

print(f"Итоговый размер размеченного датасета: {len(df_labeled)}")
print(f"Количество классов после фильтрации: {df_labeled['target'].nunique()}")

In [ ]:
# Валидация разметки
print("\n=== ВАЛИДАЦИЯ РАЗМЕТКИ (случайные примеры) ===")
sample_indices = random.sample(range(len(df_labeled)), 5)

for idx in sample_indices:
    row = df_labeled.iloc[idx]
    print(f"\nМетка: {row['target']}")
    print(f"Текст: {row['text_clean'][:200]}...")
    print("-" * 50)

### Шаг 5: Анализ размеченных данных

In [ ]:
# Статистика по классам
plt.figure(figsize=(15, 6))

# Топ-20 классов
top_classes = df_labeled['target'].value_counts().head(20)

plt.subplot(1, 2, 1)
sns.barplot(x=top_classes.values, y=top_classes.index)
plt.title('Топ-20 самых частых классов ОКПД2')
plt.xlabel('Количество примеров')

plt.subplot(1, 2, 2)
# Распределение размеров классов
class_sizes = df_labeled['target'].value_counts()
sns.histplot(class_sizes, bins=30)
plt.title('Распределение размеров классов')
plt.xlabel('Примеров в классе')

plt.tight_layout()
plt.show()

print("Статистика по классам:")
print(f"Всего классов: {len(class_sizes)}")
print(f"Медианный размер класса: {class_sizes.median()}")
print(f"Минимальный размер: {class_sizes.min()}")
print(f"Максимальный размер: {class_sizes.max()}")

In [ ]:
# Анализ длины текстов
df_labeled['text_length'] = df_labeled['text_processed'].str.split().str.len()

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.histplot(data=df_labeled, x='text_length', bins=50)
plt.title('Общее распределение длины текстов')
plt.xlabel('Длина текста (слов)')

plt.subplot(1, 2, 2)
# Длина текста по топ-10 классам
top_10_classes = class_sizes.head(10).index
df_top_classes = df_labeled[df_labeled['target'].isin(top_10_classes)]
sns.boxplot(data=df_top_classes, x='target', y='text_length')
plt.title('Длина текста по классам (топ-10)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("Статистика длины текстов:")
print(df_labeled['text_length'].describe())

### Шаг 6: Разделение на выборки

In [ ]:
from sklearn.model_selection import train_test_split

# Стратифицированное разделение
X = df_labeled['text_processed']
y = df_labeled['target']

# 70% train, 15% validation, 15% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp  # 0.176 * 0.85 ≈ 0.15
)

print("=== РАЗДЕЛЕНИЕ НА ВЫБОРКИ ===")
print(f"Обучающая выборка: {len(X_train)} записей ({len(X_train)/len(X):.1%})")
print(f"Валидационная выборка: {len(X_val)} записей ({len(X_val)/len(X):.1%})")
print(f"Тестовая выборка: {len(X_test)} записей ({len(X_test)/len(X):.1%})")

# Проверяем распределение классов в выборках
print("\nРаспределение классов по выборкам:")
for name, split in [('Train', y_train), ('Val', y_val), ('Test', y_test)]:
    print(f"{name}: {split.nunique()} классов")

### Шаг 7: Векторизация текста

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# TF-IDF с разными настройками
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),  # Учитываем отдельные слова и пары
    min_df=2,           # Игнорируем очень редкие слова
    max_df=0.9,         # Игнорируем очень частые слова
    stop_words=list(custom_stopwords)
)

# Bag-of-Words для сравнения
bow_vectorizer = CountVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.9,
    stop_words=list(custom_stopwords)
)

print("Векторизаторы созданы")

In [ ]:
# TF-IDF
print("Векторизация TF-IDF...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Bag-of-Words
print("Векторизация Bag-of-Words...")
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_val_bow = bow_vectorizer.transform(X_val)
X_test_bow = bow_vectorizer.transform(X_test)

print(f"\nРазмерности матриц признаков:")
print(f"TF-IDF: {X_train_tfidf.shape}")
print(f"BOW: {X_train_bow.shape}")

# Сохранение processed данных
df_processed = pd.DataFrame({
    'text_processed': X,
    'target': y
})
df_processed.to_csv('processed_contracts_data.csv', index=False)
print("\nОбработанные данные сохранены в 'processed_contracts_data.csv'")

---
# Чекпойнт №3: Применение простых моделей

### Шаг 1: Определение метрик качества

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

# Для многоклассовой классификации с дисбалансом
key_metric = 'f1_macro'  # F1-score (macro average)

def evaluate_model(y_true, y_pred, model_name):
    """Полная оценка модели"""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall_macro': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'f1_macro': f1_score(y_true, y_pred, average='macro', zero_division=0),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }
    
    print(f"=== {model_name} ===")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    return metrics

### Шаг 2: Baseline-модели

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

print("=== BASELINE МОДЕЛИ ===")

# KNN
print("\nОбучение KNN...")
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_tfidf, y_train)
y_pred_knn = knn.predict(X_val_tfidf)
knn_metrics = evaluate_model(y_val, y_pred_knn, "KNN")

# Logistic Regression
print("\nОбучение Logistic Regression...")
lr = LogisticRegression(random_state=42, max_iter=1000, multi_class='multinomial')
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_val_tfidf)
lr_metrics = evaluate_model(y_val, y_pred_lr, "Logistic Regression")

# Naive Bayes
print("\nОбучение Naive Bayes...")
nb = MultinomialNB()
nb.fit(X_train_bow, y_train)
y_pred_nb = nb.predict(X_val_bow)
nb_metrics = evaluate_model(y_val, y_pred_nb, "Naive Bayes")

### Шаг 3: Подбор гиперпараметров

In [ ]:
from sklearn.model_selection import GridSearchCV

print("=== ПОДБОР ГИПЕРПАРАМЕТРОВ ===")

# Оптимизация логистической регрессии
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2', 'none'],
    'class_weight': [None, 'balanced']
}

lr_grid = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000, multi_class='multinomial'),
    param_grid,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)

print("Запуск GridSearch...")
lr_grid.fit(X_train_tfidf, y_train)

print(f"Лучшие параметры: {lr_grid.best_params_}")
print(f"Лучший F1-score на кросс-валидации: {lr_grid.best_score_:.4f}")

### Шаг 4: Анализ результатов

In [ ]:
# Тестирование лучшей модели
best_model = lr_grid.best_estimator_
y_pred_best = best_model.predict(X_val_tfidf)
best_metrics = evaluate_model(y_val, y_pred_best, "Best Logistic Regression")

# Сравнение всех моделей
print("\n=== СРАВНЕНИЕ МОДЕЛЕЙ ===")
models_comparison = pd.DataFrame({
    'KNN': knn_metrics,
    'LogisticRegression': lr_metrics,
    'NaiveBayes': nb_metrics,
    'BestModel': best_metrics
})

models_comparison

In [ ]:
# Визуализация сравнения моделей
plt.figure(figsize=(12, 6))

metrics_to_plot = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro']
models_comparison.T[metrics_to_plot].plot(kind='bar', figsize=(12, 6))
plt.title('Сравнение метрик моделей')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Матрица ошибок лучшей модели
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_val, y_pred_best, labels=best_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
disp.plot(cmap='Blues', xticks_rotation=45, ax=plt.gca())
plt.title('Матрица ошибок - Best Logistic Regression')
plt.tight_layout()
plt.show()

In [ ]:
# Анализ самых частых ошибок
error_analysis = pd.DataFrame({
    'true': y_val,
    'predicted': y_pred_best,
    'text': X_val
})

errors = error_analysis[y_val != y_pred_best]
error_counts = errors.groupby(['true', 'predicted']).size().reset_index(name='count')
error_counts = error_counts.sort_values('count', ascending=False)

print("Самые частые ошибки классификации:")
print(error_counts.head(10))

print("\nПримеры ошибок:")
for i in range(min(3, len(errors))):
    row = errors.iloc[i]
    print(f"\nОшибка {i+1}:")
    print(f"Истинный класс: {row['true']}")
    print(f"Предсказанный класс: {row['predicted']}")
    print(f"Текст: {row['text'][:200]}...")

### Шаг 5: Финальные выводы

In [ ]:
print("=== ФИНАЛЬНЫЕ ВЫВОДЫ ===")

print(f"1. РАЗМЕТКА ДАННЫХ:")
print(f"   - Исходный датасет: {len(df)} записей")
print(f"   - После очистки и разметки: {len(df_labeled)} записей")
print(f"   - Количество классов: {df_labeled['target'].nunique()}")
print(f"   - Сбалансированность: медианный размер класса {class_sizes.median()}")

print(f"\n2. КАЧЕСТВО МОДЕЛЕЙ (F1-macro):")
print(f"   - KNN: {knn_metrics['f1_macro']:.4f}")
print(f"   - Naive Bayes: {nb_metrics['f1_macro']:.4f}")
print(f"   - Logistic Regression: {lr_metrics['f1_macro']:.4f}")
print(f"   - Best Model: {best_metrics['f1_macro']:.4f}")

print(f"\n3. РЕКОМЕНДАЦИИ:")
print(f"   - Лучшая модель: {type(best_model).__name__}")
print(f"   - Ключевая метрика: {key_metric}")
print(f"   - Основные проблемы: дисбаланс классов, семантически близкие классы")

# Сохранение лучшей модели
import joblib

model_artifacts = {
    'model': best_model,
    'vectorizer': tfidf_vectorizer,
    'metrics': best_metrics
}

joblib.dump(model_artifacts, 'best_model_artifacts.pkl')
print(f"\nЛучшая модель сохранена в 'best_model_artifacts.pkl'")